In [ ]:
import matplotlib.pyplot as plt
import mpl_lego as mplego
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import transformers

from pyprojroot import here
from hate_measure import keys
from hate_target import utils

from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from mpl_lego.labels import bold_text

from hate_measure.nn.classifiers import TargetIdentityClassifier
from hate_target import datasets

%matplotlib inline

In [ ]:
data_path = "~/data/unfiltered_ratings.feather"
rater_quality_path = "~/data/hatespeech/rater_quality_check.csv"
gab_path = "~/data/gab_hate_corpus.tsv"

In [ ]:
gab = pd.read_csv(gab_path, delimiter='\t')
gab['IDL_POL'] = ((gab['IDL'] == 1) | (gab['POL'] == 1)).astype('int')
gab = gab.rename({
        'ID': 'comment_id',
        'Annotator': 'labeler_id',
        'Text': 'text',
        'REL': 'target_religion',
        'RAE': 'target_race',
        'SXO': 'target_sexuality',
        'GEN': 'target_gender',
        'NAT': 'target_origin',
        'IDL_POL': 'target_politics',
        'MPH': 'target_disability'}, axis=1)

target_cols = sorted([col for col in gab.columns if 'target' in col])
gab = gab[~gab[target_cols].isna().any(axis=1)]

In [ ]:
targets = gab[['comment_id'] + target_cols]
annotator_props = targets.fillna(0).groupby('comment_id').mean()
is_target = (annotator_props >= 0.5).astype('int')
data = is_target.merge(gab[['comment_id', 'text']].drop_duplicates('comment_id'), how='left', on='comment_id')

In [ ]:
model_builder = TargetIdentityClassifier.build_model

In [ ]:
model_kwargs = {
    'transformer': 'roberta-large',
    'max_length': 247,
    'n_dense': 256,
    'dropout_rate': 0.05,
    'pooling': 'mean',
    'mask_pool': False
}

In [ ]:
model = model_builder(**model_kwargs)

In [ ]:
model.load_weights('/home/psachdeva/experiments/target_identity/exp07/exp07a_model.h5')

In [ ]:
tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-large")
tokens = tokenizer(data['text'].tolist(), return_tensors='np', max_length=247, truncation=True, padding='max_length')
inputs = [tokens['input_ids'], tokens['attention_mask']]

In [ ]:
outputs = model.predict(inputs, batch_size=256, verbose=1)

In [ ]:
y_preds = [output.ravel() for output in outputs[1:]]
y_pred_hard = [(output.ravel() >= 0.5).astype('int') for output in outputs[1:]]

In [ ]:
y_trues = [data['target_disability'].values,
           data['target_gender'].values,
           data['target_origin'].values,
           data['target_politics'].values,
           data['target_race'].values,
           data['target_religion'].values,
           data['target_sexuality'].values]

In [ ]:
[(y_true == y_pred).mean() for (y_true, y_pred) in zip(y_trues, y_pred_hard)]

In [ ]:
[1 - y_true.mean() for (y_true, y_pred) in zip(y_trues, y_pred_hard)]

In [ ]:
[f1_score(y_true, y_pred) for (y_true, y_pred) in zip(y_trues, y_pred_hard)]

In [ ]:
[roc_auc_score(y_true, y_pred) for (y_true, y_pred) in zip(y_trues, y_preds)]

In [ ]:
[average_precision_score(y_true, y_pred) for (y_true, y_pred) in zip(y_trues, y_preds)]